In [1]:
import ipywidgets as widgets
import madernpytools.madern_widgets as mwid
import madernpytools.models.toolset_model as mts
import madernpytools.backbone as mbb
import madernpytools.module_data as mdata
import numpy as np
import traitlets, inspect
import xml.etree.cElementTree as ET
import copy, os, logging, sys
import types

stream = logging.StreamHandler(sys.stdout)
stream.setFormatter(logging.Formatter("%(levelname)-8s %(message)s"))
for log_name in ['madernpytools']: #, 'ScanControl']:
    log = logging.getLogger(log_name)
    log.setLevel(logging.WARNING)
    log.addHandler(stream)


%matplotlib inline

In [2]:
btn_refresh = widgets.Button(description='Refresh Library')
lib = mts.MadernModelLibrary().load('../data/library/',verbose=False)
parts_output, parts_widgets = mwid.generate_editors_in_tab([
                                                            mts.Material, 
                                                            mts.Thread, 
                                                            mts.TensionRod, 
                                                            mts.TensionNTieRods, 
                                                            mts.RiRoTensionScrew,
                                                            mts.SetScrew, 
                                                            mts.SpacerNSetScrews, 
                                                            mts.RiRoShimPack
                                                            
                                                           ],
                                                          lib=lib)
bearings_output, bearing_widgets = mwid.generate_editors_in_tab([mts.SphericalBearing, 
                                                                 mts.TaperedBearing, 
                                                                 mts.OTaperedBearing], lib=lib)


ts_output,  ts_widgets = mwid.generate_editors_in_tab([mts.BobstToolset, mts.Cylinder, 
                                                       mts.BearingBlock,
                                                       mts.BearerRing, mts.Shaft, 
                                                       mts.BearingBlock, mts.SimpleLayout, mts.Gear], lib=lib)

#btn_refresh.on_click(lambda obj: mwid.refresh_library({**ts_widgets, **parts_widgets, **bearing_widgets},
#                                                      mts.MadernModelLibrary().load('./data/library/')
#                                                     )
#                    )
#
# Accordeon:
my_accordion = widgets.Accordion(children=[bearings_output, parts_output, ts_output])
for i, name in enumerate(['Bearings', 'Toolset Parts', 'Toolset Configuration']):
    my_accordion.set_title(i, name)


#my_accordion.titles = ['Bearings', 'Toolset Parts', 'Toolset Configuration']
#my_accordion._title(0, 'Bearings')
#my_accordion.set_title(1, 'Toolset Parts')
#my_accordion.set_title(2, 'Toolset Configuration')
my_accordion.selected_index = None

display(btn_refresh)
display(my_accordion)


Button(description='Refresh Library', style=ButtonStyle())

Accordion(children=(Tab(children=(ModelEditor(), ModelEditor(), ModelEditor()), _titles={'0': 'SphericalBearin…

In [ ]:
my_accordion.set_title()

In [ ]:
lib.get_of_type(mts.IToolset)

In [ ]:
tie_rod = lib.get_of_type(mbb.MadernObject)['DR67-tierod (v2016)']
spacer = lib.get_of_type(mbb.MadernObject)['bobst_DR67_2setscrew_spacer']
bearing = lib.get_of_type(mbb.MadernObject)['bearing_32016']
bearing_block = mts.BearingBlock(bearing=bearing, s=50, mass=60.0)
layout = mts.SimpleLayout(ax_line_length=600)
upper_gear = mts.Gear(width=50, d_pitch=220, d_bore=80, material=mts.steel, module=2) 
lower_gear = mts.Gear(width=50, d_pitch=220, d_bore=80, material=mts.steel, module=2) 
        
upper_cylinder = mts.Cylinder(
                        os_shaft=mts.Shaft(inner_diameter=0, outer_diameter=80,material=mts.steel, length=100),
                        ds_shaft=mts.Shaft(inner_diameter=0, outer_diameter=80,material=mts.steel, length=100),
                        body=mts.Shaft(inner_diameter=0, outer_diameter=200,material=mts.steel, length=800),
                       bearer_ring=mts.BearerRing(width=50,diameter=200,angle=1.0,material=mts.steel,thickness=50),
                       br_location=300)

lower_cylinder = mts.Cylinder(
                        os_shaft=mts.Shaft(inner_diameter=0, outer_diameter=80,material=mts.steel, length=100),
                        ds_shaft=mts.Shaft(inner_diameter=0, outer_diameter=80,material=mts.steel, length=100),
                        body=mts.Shaft(inner_diameter=0, outer_diameter=200,material=mts.steel, length=850),
                       bearer_ring=mts.BearerRing(width=50,diameter=200,angle=1.0,material=mts.steel,thickness=50),
                       br_location=300)

ts = mts.BobstToolset(upper_cylinder=upper_cylinder,
                      lower_cylinder=lower_cylinder,
                      spacer=spacer,
                      tensioner=tie_rod,
                      upper_bearing_block=copy.copy(bearing_block),
                      lower_bearing_block=copy.copy(bearing_block),
                      upper_gear = upper_gear,
                      lower_gear = lower_gear,
                      layout=layout,
                      name='Toolset',
                      F_t = 20e3,
                      q_cut= 0.0
                     )
        
        
w_ts = mwid.ToolsetWidget(ts, lib) 
display(w_ts)

In [ ]:
def cb_save_btn(self):
    # Collect filename:
    fn = mwid.get_file_name()
    if fn != '':
        fn = os.path.splitext(fn)[0]
        ET.ElementTree(ts.to_xml()).write('{}.xml'.format(fn))
        
save_btn = widgets.Button(description='Save')
save_btn.on_click(cb_save_btn)
display(save_btn)

In [ ]:
mwid.MadernPropertyDisplay()

In [ ]:
import abc
def NotImplementedField(self):
    raise NotImplementedField()
    
class AbstractClass(abc.AbstractClass):
    
    @property
    @abc.abstractmethod
    def x(self):
        raise NotImplementedError()
    
class FailTest(AbstractClass):
    pass
    
class Test(AbstractClass):
    x='bla'
    

FailTest()
